In [1]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
var('xx')
Q.<ii> = NumberField(xx^2 + 1)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [4]:
# Define cubic SE1

In [5]:
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [6]:
# Find conditions associated to L-set ('E1', 'G4', 'E3', 'G6', 'F24').

In [7]:
%time _, all_ideals = SE1.find_conditions_on_L_sets([('E1', 'G4', 'E3', 'G6', 'F24')], "eck")
current_ideal = all_ideals[0] if all_ideals[0].gens()[0].coefficient(d) == -ii else all_ideals[1]
current_ideal

CPU times: user 3.92 s, sys: 8.05 ms, total: 3.93 s
Wall time: 3.92 s


Ideal (4*c + (-ii)*d + (ii)*e + (-ii + 2)*f, 5*d^2 + (-4*ii - 2)*d*e + (4*ii - 3)*e^2 + (8*ii - 6)*d*f + (36*ii + 38)*e*f + (-24*ii - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1

In [8]:
# Find substitution for d and e parameters

In [9]:
poly_e = current_ideal.gens()[0]
assert poly_e.degree(e) == 1
e_sost = {e: find_linear_solution(poly_e, e)}
poly_d = [gen for gen in current_ideal.subs(e_sost).gens() if gen !=0][0]
assert poly_d.degree(d) == 1
d_sost = {d: find_linear_solution(poly_d, d)}  
# Substitute d sostitution into e to obtain sostitution with only c and f as parameters
e_sost = {e: e_sost.get(e).subs(d_sost)}
print("d substitution: d =", d_sost.get(d).factor(), "\ne substitution: e =", e_sost.get(e).factor())

d substitution: d = ((-2*ii - 1)) * (c - f)^-1 * (c^2 + (-4/5*ii - 8/5)*c*f - f^2) 
e substitution: e = ((2*ii - 1)) * (c - f)^-1 * c * (c + (-4/5*ii + 3/5)*f)


In [10]:
# Extract substitution to obtain cubic surfaces with bigger simmetry group.

In [11]:
%time SE1_1 = SE1.subs(e_sost).subs(d_sost)

CPU times: user 34.1 s, sys: 11.8 ms, total: 34.1 s
Wall time: 34.1 s


In [12]:
# Find simmetries

In [13]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 3.42 s, sys: 186 ms, total: 3.6 s
Wall time: 41.8 s


576

In [14]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 2.13 s, sys: 186 ms, total: 2.31 s
Wall time: 13.1 s


4

In [15]:
# Study how the simmetries permute the Eckardt points

In [16]:
Eck_perms = [Permutation(SE1_1.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE1_1]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(1, '1')

In [17]:
### geometric interpretation of the group of symmetries

In [18]:
%%time
## Action of the stabilizer of SE1' on the 27 lines:
Ga = perm_group_lines(SE1_1, simm_SE1_1)

## Action of the stabilizer of SE1' on the 45 tritangent planes:
Gb = perm_group_planes(SE1_1, simm_SE1_1)

CPU times: user 3.54 s, sys: 52.4 ms, total: 3.59 s
Wall time: 3.61 s


In [19]:
print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
C4

Structure of the group Gb:
C4


In [20]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [21]:
# The only lines fixed are those meeting in the Eckardt point
for orb in OGa:
    if len(orb) == 1:
        print(simbolic_line_from_index(orb[0]))

E1
G4
F14


In [23]:
# The only tritangent plane fixed is tau_3
for orb in OGb:
    if len(orb) == 1:
        fixed_plane = simbolic_plane_from_index(orb[0])
        print(fixed_plane, tritangent_plane_from_name(fixed_plane))

t3 ['E1', 'G4', 'F14']
